**Imports**

Data

In [ ]:
#enter the path to get to your CDs_and_Vinyl folder below
PATH_TO_CDS = '../devided_dataset_v2/CDs_and_Vinyl/'

import pandas as pd

# Open and load json training files
x = pd.read_json(PATH_TO_CDS + 'train\\review_training.json')
y = pd.read_json(PATH_TO_CDS + 'train\\product_training.json')

Other

In [ ]:
import numpy as np
from nltk import sentiment

PATCH TO SPEED UP (ONLY IF INTEL CHIP) 

lmao rip daniel

In [ ]:
from sklearnex import patch_sklearn 

patch_sklearn()

**Pre Processing**

In [ ]:
#z = pd.merge(x,y,'inner','asin')
#zgrouped = z.groupby("asin")

In [ ]:
x #770786 entries
y #073082 entries

Feature creation testing

Making a single feature ex

In [ ]:
reviewCount = x.groupby('asin')["reviewerID"].count()
reviewCount

In [ ]:
unixReviewMean = x.groupby('asin')['unixReviewTime'].mean()
unixReviewMean

Dropping ex

In [ ]:
dropUnverified = x[x.verified != True]
dropUnverified

groupby() testing

In [ ]:
tests = range(5)
unique_items = z["asin"].nunique()
print("ui: ", unique_items)

asin = z["asin"].values
print("asin: ",asin)
avg_votes = np.zeros(len(asin))

for i in tests: # eventually all
    item = zgrouped.get_group(asin[i])
    avg_votes[i] = item["vote"].fillna(0).astype(int).mean()

print("avg_votes: ", avg_votes[0:5])

# Sentiment Analysis

In [ ]:
## Helper function for sentiment
def review_sentiment(text):
    sia = sentiment.SentimentIntensityAnalyzer()
    return sia.polarity_scores(text)["compound"]

## just some examples to gut check
print(review_sentiment("Ugh, what a terrible, horrible, no good, very bad day"))
print(review_sentiment("I went to the store today"))
print(review_sentiment("I absolutely love my favorite best friend"))

apply sentiment analysis to data

In [ ]:
tests = range(1)
unique_items = z["asin"].nunique()
print("ui: ", unique_items)

asin = z["asin"].unique()
# print("asin: ",asin[tests])
avg_votes = np.zeros(len(asin))

for i in tests: # eventually all
    item = zgrouped.get_group(asin[i])
    avg_votes[i] = item["vote"].fillna(0).astype(int).mean()
    print(len(item))
    sentiments = map(review_sentiment,item["reviewText"].values[0:1000])
    
    print(list(sentiments))

print("avg_votes: ", avg_votes[0:5])

Test example: 

*Gaussian Naive Bayes with tfidf*

In [ ]:
from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

#split data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=0)

#train and apply tfidf
    # fit = make vocab 
    # transform = turn words -> numbers
x_train = tfidf.fit_transform(x_train).toarray()
x_test = tfidf.transform(x_test).toarray()

#make prediction
y_pred = gnb.fit(x_train, y_train).predict(x_test)

#print accuracy
print(f"Number of mislabeled points out of a total {x_test.shape[0]} points : {(y_test != y_pred).sum()}")

# Models

(choose one)

Basic Support Vector Machine (SVM)

In [ ]:
from sklearn import svm

model = svm.SVC(kernel='rbf')

Basic Gaussian Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()

Decision Tree

In [ ]:
from sklearn import tree

#choose from 'entropy', 'gini', 'log_loss'
criterion = 'entropy'

    ###STOPPING CONDITIONS
#max height/depth of the tree
max_depth = 4

#if sat(N) < n_samples*this, don't try to split it any further
min_samples_split = .01

    ###OTHER
#if min(sat(nodes after a split)) < n_samples*this, split not considered 
#       (aka model isnt allowed to create leaf nodes with < n_samples*this samples)
#       (smooths the model, avoids splits like 2000 -> 1999 vs 1)
min_samples_leaf = .01

model = tree.DecisionTreeClassifier(criterion=criterion, max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)

For reference: What x,y should look like

In [ ]:
from sklearn.datasets import load_iris

iris = load_iris()

#iris.data
#iris.target

# Testing prep

Put feature vector with col 1 as 'asin' to *z*, answers to *y*

In [ ]:
#combine all individual features into one dataFrame

#enter any features to be combined here! 
#   They must be pd dataFrames with the 'asin' column for this to work
features = [reviewCount,unixReviewMean]

z = x['asin']
for f in features:
    z = pd.merge(z,f,'inner','asin')

#combine resultant data with correct answers 
temp = pd.merge(z,y,'inner','asin')

#split into features (x) and awesomeness (y), which now row correspond but don't have an 'asin' col anymore
merged_x = temp.drop(['asin','awesomeness'], axis=1)
merged_y = temp["awesomeness"]

Decision Tree - See what it looks like! 

(only run if the model is a decision tree)

*note: this trains the model on the whole dataset, so don't do before testing!*

In [ ]:
model.fit(merged_x,merged_y)
tree.plot_tree(model)

# Testing

K-Fold testing 

(set model to variable *model*)

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn import metrics

#desired metrics
scoring=['f1','accuracy','precision','recall']

#number of "folds"
k=10

cv_results = cross_validate(model, merged_x, merged_y, cv=k, scoring=scoring)

Pretty print results

In [ ]:
hline = ("-"*40)

#print means, stdevs
print(hline+"\n\tAVG, STDEV TEST RESULTS\n"+hline)
print("\t\tMEAN\t\tSTDEV")
for m in scoring:
        ind = ("test_"+m)
        test = (m+":").ljust(10)
        print(f"{test}\t{np.round(cv_results[ind].mean(),4)}\t\t{np.round(cv_results[ind].std(),4)}")


    ##print per test results (if you really want, uncomment)
# print("\n")
# print(hline+"\n\tPER TEST RESULTS\n"+hline)
# for i in range(k):
#     print(f"Test {i}:")
#     for m in scoring:
#         ind = ("test_"+m)
#         test = (m+":").ljust(10)
#         print(f"\t{test} {np.round(cv_results[ind][i],5)}")